# Example of an RNN

We construct a simple RNN to generate text from a work of Nietsche

## Setup libraries and functions.

We start by importing various libraries needed.

In [3]:
# Plots displayed inline in notebook
%matplotlib inline

# Make Python 3 consistent
from __future__ import print_function, division

# Make help libraries available
import sys

sys.path.append('/home/ubuntu/personal-libraries')

In [41]:
import numpy as np

from keras.models import Sequential
from keras.layers import Embedding, LSTM, TimeDistributed, Activation
from keras.layers.core import Dropout, Dense
from keras.optimizers import Adam
from keras.utils import get_file

from keras import backend as K
K.set_image_data_format('channels_first')

## Setup data

Import the Nietsche data and save it as text.

In [5]:
path = get_file('nietzsche.txt',
                origin = "https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()

print('corpus length:', len(text))

475136/600901 [======================>.......] - ETA: 0scorpus length: 600901


Let's consider an example. We see that the text is very convoluted (haha), which in this case make it easier for a model, as it will be hard for us humans to undertand wether something is true Nietsche tect.

In [6]:
!tail {path} -n25

are thinkers who believe in the saints.


144

It stands to reason that this sketch of the saint, made upon the model
of the whole species, can be confronted with many opposing sketches that
would create a more agreeable impression. There are certain exceptions
among the species who distinguish themselves either by especial
gentleness or especial humanity, and perhaps by the strength of their
own personality. Others are in the highest degree fascinating because
certain of their delusions shed a particular glow over their whole
being, as is the case with the founder of christianity who took himself
for the only begotten son of God and hence felt himself sinless; so that
through his imagination--that should not be too harshly judged since the
whole of antiquity swarmed with sons of god--he attained the same goal,
the sense of complete sinlessness, complete irresponsibility, that can
now be attained by every individual through science.--In the same manner
I have viewed the saints of India

We then make a list of all charcters used in the text. And determine the size of the vocabulary.

In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars) + 1
print('total chars:', vocab_size)

total chars: 60


We add a character by hand. And consider the output (-6 removes forreign utf-8 characters).

In [13]:
chars.insert(0, "\0")

In [14]:
''.join(chars[1:-6])

'\x00\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyz'

We then make a dict for characters and for indices. So each character gets mapped to an indecy

In [15]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

We then convert the text using these indices.

In [20]:
idx = [char_indices[c] for c in text]

Consider the first ten characters.

In [23]:
idx[:10]

[44, 46, 33, 34, 29, 31, 33, 2, 2, 2]

Corresponding to (Note the annoying line feeds)

In [24]:
''.join(indices_char[i] for i in idx[:10])

'preface\n\n\n'

## Preprocess and create model

We then construct running sentences where each sentence in a pair is one character aahead of the other sentence in the pair.

In [27]:
maxlen = 40
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen + 1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i + maxlen + 1])
print('nb sequences:', len(sentences))

nb sequences: 600862


We can then construct an array of sentences and the same sentences moved one character forward. Essentially a training set and a label set.

In [30]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

We can validate that the two arrays have the same shape. That is 600860 sentences of 40 characters.

In [31]:
sentences.shape, next_chars.shape

((600860, 40), (600860, 40))

We speccify the number of factors to use in our embedding of the sentences.

In [33]:
n_fac = 24

An we can then contruct out RNN, specifically a long-short term memory recurrent neural network.

In [60]:
model = Sequential([
        Embedding(vocab_size, n_fac, input_length = maxlen),
        LSTM(512,
             input_shape = (None, n_fac),
             return_sequences = True,
             recurrent_dropout = 0.2,
             dropout = 0.2,
             implementation = 2),
        Dropout(0.2),
        LSTM(512,
             return_sequences = True,
             recurrent_dropout = 0.2,
             dropout = 0.2,
             implementation = 2),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])    

We compile the model

In [55]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = Adam())

## Train the LSTM RNN

We define a function such as to print sample predictions.

In [56]:
def print_example():
    
    # WE have to see the model, so we pick something, that sounds Nietsche like.
    seed_string = "ethics is a basic foundation of all that"
    
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose = 0)[0][-1]
        preds = preds / np.sum(preds)
        next_char = choice(chars, p = preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [57]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 40, 24)            1440      
_________________________________________________________________
lstm_9 (LSTM)                (None, 40, 512)           1099776   
_________________________________________________________________
dropout_9 (Dropout)          (None, 40, 512)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 40, 512)           2099200   
_________________________________________________________________
dropout_10 (Dropout)         (None, 40, 512)           0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 40, 60)            30780     
_________________________________________________________________
activation_3 (Activation)    (None, 40, 60)            0         
Total para

We then train the model for a single epoch

In [58]:
model.fit(sentences, np.expand_dims(next_chars, -1), batch_size = 64, epochs = 1)

Epoch 1/1
   768/600860 [..............................] - ETA: 1572s - loss: 3.6519

IndexError: One of the index value is out of bound. Error code: 65535.\n
Apply node that caused the error: GpuAdvancedSubtensor1(embedding_6/embeddings, Elemwise{Composite{Cast{int64}(Cast{int32}(i0))}}.0)
Toposort index: 171
Inputs types: [CudaNdarrayType(float32, matrix), TensorType(int64, vector)]
Inputs shapes: [(60, 24), (2560,)]
Inputs strides: [(24, 1), (8,)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[GpuReshape{3}(GpuAdvancedSubtensor1.0, MakeVector{dtype='int64'}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [53]:
np.expand_dims(next_chars, -1).shape

(600860, 40, 1)

And print the prediction. After the first epoch we see that sentence and word structure is starting to form, but the output is rather terrible.

In [220]:
print_example()

ethics is a basic foundation of all thatscrriets sdi  ,s lrrbmh
fceelsora tec
 n yiefma
cnostencnrut -  o
pen.htt" oaiosovo  stialpts es rb b
ea ie
ohatnmauyielueysiutlmo,es  etfrne oh
ohnio iis e.eosme o rdorfdbteirnse ohdnotafi enicron e eietnyn sytt e ptsrdrede httmi ah
oo, tdye es r,igyct toehitu abrh ei isiem-r natra lnspamlltefae a
cni vuui
twgt fatieh


So we run for an epoch more.

In [236]:
model.fit(sentences, np.expand_dims(next_chars, -1), batch_size = 64, eochs = 1)

Epoch 1/1
600860/600860 [==============================] - 640s - loss: 1.5152   


Now it's actually starting to construct words, sentences and using punctuation. But it is still rather meaningsless.

In [222]:
print_example()

ethics is a basic foundation of all that he maluces indofely and is; pticrast', and re onerog je ivesantamale as whered
and ror and kytinf? on chaninn nurdeln--ans prory. heke the pepadinar; anf bom,
puntely"" ones to bucf, alcherstol the qisleves: the the wite dadong the gur is prang not galcaula rewinl
more by than sic appads not pepow o mee, a more
bins c


So we decrease the learning rate and run for another epoch

In [236]:
model.optimizer.lr = 0.001

odel.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 640s - loss: 1.5152   


Now we're starting to get somewhere. We have somthing that can be read and is actual words.

In [237]:
print_example()

ethics is a basic foundation of all that schools pedhaps a new prisons of the ashamed in which
a coverbine estimates of the assumption that one avoid; he will curse about pain:
     people, he-equally present to
the lalier,
nature. that he has
rendered and henceforth distrain and impulses to perceive that each other
former and dangerous, and cannot at
the pu


We decrease the learning rate and run another epoch.

In [239]:
model.optimizer.lr = 0.0001

model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 639s - loss: 1.2892   


Now we are starting to get chapter marks and paragraph marks as well.

In [240]:
print_example()

ethics is a basic foundation of all that account has its granitify them.

131. the new "dilence," out of the
same light,
interpretation thereof: under the "thinking"
there, to counter-arguments in the monality, so many language:
though
all nobilitys of higher impulses, man and hence to everything of seldom man.



chapter i. woman decides according the injur


So we save the weights.

In [242]:
model.save_weights('data/char_rnn.h5')

And decrease the learning rate even more and run for another epoch.

In [257]:
model.optimizer.lr = 0.00001

In [243]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 640s - loss: 1.2544   


Now we are getting something quite nietsche like.

In [ ]:
print_example()

Let's run a final epoch and consider two examples. Very Nietsche like now.

In [258]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 640s - loss: 1.2218   


In [264]:
print_example()

ethics is a basic foundation of all that
the belief in the importance. the employs concerning
seriousness and
materialism, it is circles which alone is already attained, that he sees
also the day after thinking
of mankind, brightness, resistance--and after the value of "nature" in order to nevertheless
have taken a system of liberal fatalists are willing him


In [283]:
print_example()

ethics is a basic foundation of all that were beought by the temptation of truth--for the rest of a sublime medely and take part of life, which lacks himself the
credibility about this, in short, and raise such a
gods; and on the
other hand, the explanation of
the case, as the most ingredient, and insight, and approach as to the
peculiarly prolonged "distrus


In [282]:
model.save_weights('data/char_rnn.h5')